In [15]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os
from litellm import completion
import re

In [16]:
with open("../../../api_key_aipm.txt", 'r') as file:
    OPENAI_API_KEY = file.read().strip()
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [17]:
def generate_prompt(original_text, context):
    """
    Generate personalized text using an LLM model.

    Args:
        original_text (str): The original text from the HTML element.
        context (dict): A dictionary containing user-specific information or preferences.

    Returns:
        str: The generated text.
    """
    # Example prompt for the model
    prompt = f"""
    You are a highly creative and professional text generator for websites. 
    Your task is to rewrite the following text to make it more engaging and tailored 
    to the following user context: {context}.
    
    Original text: "{original_text}"
    
    Rewrite the text in a way that is personalized and keeps its original intent. 
    If the information is not changeable, like an adress or a date, don't change anything and return the original text.

    Please return the output as clean text and keep the output approximately the same length as the original text.
    """

    return prompt

def get_new_string(original_text, context, model="o1-mini", **kwargs):
    prompt = generate_prompt(original_text, context)
    messages = [{"role": "user", "content": prompt}]
    
    response = completion(
        messages=messages,
        model=model,
        **kwargs
    )

    new_string = response.choices[0].message.content
    new_string = re.sub(r"[*•<>]", "", new_string)  # Remove asterisks, bullet points, or HTML-like tags
    new_string = new_string.strip() 
    
    return new_string

In [18]:
def generate_new_html(url, html_elements, output_name, context): 
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    for s in soup.find_all(html_elements):
        if s.string and s.string != " ":
            new_string = get_new_string(s.string, context)
            s.string.replace_with(new_string)

    with open(output_name, "w") as file:
        file.write(str(soup))

In [19]:
url = "https://www.lausanne-palace.ch/accueil/"
html_elements = ["h1", "h2", "p", "h4", "h3"]
output_name = "bs4_output.html"
context = 'Appeal a younger audience'

In [20]:
generate_new_html(url, html_elements, output_name, context)

In [21]:
example = "Chaque jeudi soir, plongez dans l’atmosphère élégante et intime du LP’s Bar et laissez-vous envoûter par notr programmation en live. Que vous soyez passionné de jazz ou en quête d’une expérience musicale sophistiquée, profitez d’un moment de détente idéal entre amis ou collègues."
get_new_string(example, context)

'Tous les jeudis soir, découvrez l’ambiance tendance et conviviale du LP’s Bar et laissez-vous emporter par notre programmation live. Que vous soyez fan de jazz ou en quête d’une expérience musicale moderne, profitez d’un moment fun entre amis ou collègues.'